# January 27-30, 22

In [1]:
import os
import sys
from os.path import join as pjoin


import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import pickle 

import shap

# main dirs
proj_dir = pjoin(os.environ['HOME'], 'explainable-ai')

# folders
sys.path.insert(0, proj_dir)
from helpers.dataset_utils import *
from helpers.base_model import *
from helpers.model_definitions import *

# select the GPU to be used
gpus = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_memory_growth(gpus[1], True)
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

/home/govindas/venvs/expln-ai3.9/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:37: UserWarning: You are currently using a nightly version of TensorFlow (2.9.0-dev20220124). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


## model

In [2]:
model = Linear_Model()

default_slice = lambda x, start, end : x[start : end, ...]

linear_regression = base_model(task_type="regression", 
                               model=model, 
                               loss_object=tf.keras.losses.MeanSquaredError(), 
                               L1_scale=0.0, 
                               L2_scale=0.0,
                               optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                               eval_metric=tfa.metrics.RSquare(),
                               eval_metric_name="% var explained",
                               batch_size=32, 
                               slice_input=default_slice)

2022-01-29 18:54:32.032533: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-29 18:54:32.475220: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14796 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5


## data

In [3]:
# get data
with open(pjoin(proj_dir, 'data/emoprox2', 'train_test_arrays.pkl'), 'rb') as f:
    data_dict = pickle.load(f)

# converting to tf tensors
data_dict['train'] = to_tensor(data_dict['train'])
data_dict['test'] = to_tensor(data_dict['test'])

# get inputs, targets and masks
train_X = data_dict['train'][0]
train_y = data_dict['train'][1]
train_mask = data_dict['train'][2]

test_X = data_dict['test'][0]
test_y = data_dict['test'][1]
test_mask = data_dict['test'][2]

# mask the tensors
train_X = train_X * tf.expand_dims(tf.cast(train_mask, 'float32'), -1)
train_y = train_y * tf.cast(train_mask, 'float32')

test_X = test_X * tf.expand_dims(tf.cast(test_mask, 'float32'), -1)
test_y = test_y * tf.cast(test_mask, 'float32')

In [4]:
tf.expand_dims(tf.cast(train_mask, 'float32'), -1).shape

TensorShape([647, 360, 1])

## train the model

In [9]:
results = linear_regression.fit(train_X=train_X, 
                                train_Y=train_y, 
                                val_X=train_X, 
                                val_Y=train_y, 
                                num_epochs=15)

Epoch 000: Train Loss: 0.065, Train % var explained: 5.607%  Val Loss: 0.064, Val % var explained: 6.339%  
Epoch 001: Train Loss: 0.064, Train % var explained: 6.611%  Val Loss: 0.064, Val % var explained: 7.024%  
Epoch 002: Train Loss: 0.064, Train % var explained: 6.995%  Val Loss: 0.063, Val % var explained: 7.224%  
Epoch 003: Train Loss: 0.064, Train % var explained: 7.090%  Val Loss: 0.063, Val % var explained: 7.295%  
Epoch 004: Train Loss: 0.064, Train % var explained: 7.097%  Val Loss: 0.063, Val % var explained: 7.291%  
Epoch 005: Train Loss: 0.064, Train % var explained: 7.055%  Val Loss: 0.063, Val % var explained: 7.257%  
Epoch 006: Train Loss: 0.064, Train % var explained: 6.969%  Val Loss: 0.063, Val % var explained: 7.207%  
Epoch 007: Train Loss: 0.064, Train % var explained: 6.822%  Val Loss: 0.063, Val % var explained: 7.152%  
Epoch 008: Train Loss: 0.064, Train % var explained: 6.582%  Val Loss: 0.064, Val % var explained: 7.097%  
Epoch 009: Train Loss: 0.064

## Shapley values